In [1]:
import os
import pandas as pd
import glob

iemocap_path = '/iemocap/IEMOCAP_full_release'
sessions = [f'Session{i}' for i in range(1, 6)]
data = []

# 4-class labels
allowed_emotions = {'ang', 'hap', 'sad', 'neu', 'exc'}

for session in sessions:
    eval_dir = os.path.join(iemocap_path, session, 'dialog', 'EmoEvaluation')
    trans_dir = os.path.join(iemocap_path, session, 'dialog', 'transcriptions')
    wav_base_dir = os.path.join(iemocap_path, session, 'sentences', 'wav')

    for eval_file in os.listdir(eval_dir):
        if not eval_file.endswith('.txt'):
            continue
        with open(os.path.join(eval_dir, eval_file), 'r') as f:
            lines = f.readlines()
            for line in lines:
                if line.startswith('['):
                    parts = line.strip().split('\t')
                    if len(parts) >= 3:
                        utt_id = parts[1]
                        emotion = parts[2]
                        
                        if emotion not in allowed_emotions:
                            continue

                        # Merge 'exc' into 'hap'
                        if emotion == 'exc':
                            emotion = 'hap'

                        dialog_id = '_'.join(utt_id.split('_')[:3])  # e.g., Ses05F_script01_1
                        trans_file_pattern = os.path.join(trans_dir, f'{utt_id.split("_")[0]}_{utt_id.split("_")[1]}_*.txt')

                        # Search for matching transcription files
                        matches = sorted(glob.glob(trans_file_pattern))
                        if matches:
                            trans_file = matches[0]  # Use the first match
                            
                            # Determine type: 'scripted' or 'improvised'
                            utt_type = 'scripted' if 'script' in dialog_id.lower() else 'improvised'

                            with open(trans_file, 'r') as tf:
                                for tline in tf:
                                    if utt_id in tline:
                                        text = tline.strip().split(': ')[-1]
                                        wav_path = os.path.join(wav_base_dir, dialog_id, utt_id + '.wav')

                                        data.append({
                                            'utterance_id': utt_id,
                                            'dialog_id': dialog_id,
                                            'emotion': emotion,
                                            'text': text,
                                            'wav_path': wav_path,
                                            'type': utt_type
                                        })
                                        break

# Create DataFrame
df = pd.DataFrame(data)
df = df.sort_values(by='utterance_id').reset_index(drop=True)

# Save to CSV
output_csv_path = '/IEMOCAP_full_release/iemocap_4.csv'
df.to_csv(output_csv_path, index=False)
print(f"✅ Saved filtered 4-class data with full dialog_id and utterance type to: {output_csv_path}")


✅ Saved filtered 4-class data with full dialog_id and utterance type to: /iemocap/IEMOCAP_full_release/iemocap_4.csv
